## Just Check Data

In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
import mmcv

import pickle
import torch
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from pprint import pprint
from tqdm import tqdm

/workspace/shanoa/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/shanoa/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [39]:
dir = 'Nancho_dataset/defV2/thr_range[0_2]/' #swin_nancho_deformable2
model_name = 'swin_defor.py' #scales/swin_defor, 'top_cross_val_swin_nancho_defor.py'
# dir= 'swin_nancho'
# model_name= 'swin_nancho.py'
model_load = 'mmdetection/work_dirs/'+dir+'/epoch_20.pth' #/scales, /original/best_mAP_epoch_6
# model_load = 'mmdetection/work_dirs/'+'S03/swin_t'+'/base_best_mAP_epoch_8.pth' #usa S03 as model

root_dir = '/workspace/mmdetection_mau/data/Nancho_dataset/'  #+dir+'/'  #Nancho_dataset, kuzushiji
config_model = 'mmdetection/work_dirs/'+dir+'/'+model_name 
save_path = '/workspace/mmdetection_mau/code/mmdetection/work_dirs/'+dir+'/' #/scales/, /original/

In [40]:
# https://github.com/matterport/Mask_RCNN/issues/2408
# https://gist.github.com/aunsid/b28c87f98983f00163f6e588e3da1191
## Some extra funtions
def get_iou(a, b, epsilon=1e-5, intersection_check=False):
    x1 = max(a[0], b[0])
    y1 = max(a[1], b[1])
    x2 = min(a[2], b[2])
    y2 = min(a[3], b[3])
    score = b[4]
    
    width = (x2 - x1)
    height = (y2 - y1)

    if (width < 0) or (height < 0):
        if intersection_check:
            return 0.0, False
        else:
            return 0.0
    area_overlap = width * height

    area_a = (a[2] - a[0]) * (a[3] - a[1])
    area_b = (b[2] - b[0]) * (b[3] - b[1])
    area_combined = area_a + area_b - area_overlap

    iou = area_overlap / (area_combined + epsilon)
    if intersection_check:
        return iou, bool(area_overlap)
    else:
        return iou

###todo: get mAP after plotting adding the info in the df
def average_precision(recalls, precisions, mode='area'):
    """Calculate average precision (for single or multiple scales).

    Args:
        recalls (ndarray): shape (num_scales, num_dets) or (num_dets, )
        precisions (ndarray): shape (num_scales, num_dets) or (num_dets, )
        mode (str): 'area' or '11points', 'area' means calculating the area
            under precision-recall curve, '11points' means calculating
            the average precision of recalls at [0, 0.1, ..., 1]

    Returns:
        float or ndarray: calculated average precision
    """
    no_scale = False
    if recalls.ndim == 1:
        no_scale = True
        recalls = recalls[np.newaxis, :]
        precisions = precisions[np.newaxis, :]
    assert recalls.shape == precisions.shape and recalls.ndim == 2
    num_scales = recalls.shape[0]
    print("class: ",num_scales )
    ap = np.zeros(num_scales, dtype=np.float32)
    if mode == 'area':
        zeros = np.zeros((num_scales, 1), dtype=recalls.dtype)
        ones = np.ones((num_scales, 1), dtype=recalls.dtype)
        mrec = np.hstack((zeros, recalls, ones))
        mpre = np.hstack((zeros, precisions, zeros))
        for i in range(mpre.shape[1] - 1, 0, -1):
            mpre[:, i - 1] = np.maximum(mpre[:, i - 1], mpre[:, i])
        for i in range(num_scales):
            ind = np.where(mrec[i, 1:] != mrec[i, :-1])[0]
            ap[i] = np.sum(
                (mrec[i, ind + 1] - mrec[i, ind]) * mpre[i, ind + 1])
    elif mode == '11points':
        for i in range(num_scales):
            for thr in np.arange(0, 1 + 1e-3, 0.1):
                precs = precisions[i, recalls[i, :] >= thr]
                prec = precs.max() if precs.size > 0 else 0
                ap[i] += prec
        ap /= 11
    elif mode == '101points':
        for i in range(num_scales):
            for thr in np.arange(0.5, .95 , 0.05):
                precs = precisions[i, recalls[i, :] >= thr]
                prec = precs.max() if precs.size > 0 else 0
                ap[i] += prec
        ap /= 101
    else:
        raise ValueError(
            'Unrecognized mode, only "area" and "11points" are supported')
    if no_scale:
        ap = ap[0]
    return ap

def calc_conditions(id, gt_boxes, pred_boxes, iou_thresh=0.5, hard_fp=True):
    gt_class_ids_ = np.zeros(len(gt_boxes))
    pred_class_ids_ = np.zeros(len(pred_boxes))

    tp, fp, fn = 0, 0, 0
    fppc = det_rate = 0
    for i in range(len(gt_class_ids_)):
        iou = []
        for j in range(len(pred_class_ids_)):
            now_iou, intersect = get_iou(gt_boxes[i], pred_boxes[j], intersection_check=True)
            if now_iou >= iou_thresh and intersect:
                iou.append(now_iou)
                gt_class_ids_[i] = 1
                pred_class_ids_[j] = 1
        if len(iou) > 0: 
            tp += 1 
            fp += len(iou) - 1
    fn += np.count_nonzero(np.array(gt_class_ids_) == 0)
    fp += np.count_nonzero(np.array(pred_class_ids_) == 0)
    
    if tp > 0:
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1_score = 2 * ((precision * recall) / (precision + recall))
        iou = tp / (tp + fp + fn)
        fppc = fp / (tp+fn) #(tp + fp - (tp))/(tp+fn)
        det_rate= tp / (tp + fn) #accuracy tp/(tp+fp+fn) 
        fppc= float("{0:.5f}".format(fppc)) #False positives per character
        det_rate= float("{0:.5f}".format(det_rate)) #detection rate (accuracy)
        
    else:
        fppc = fp / (tp+fn) #(tp + fp - (tp))/(tp+fn)
        fppc= float("{0:.5f}".format(fppc)) #False positives per character
        precision = recall = f1_score = iou = det_rate  = 0.0

    return {'image_id': id, 'fppc':fppc, 'det rate':det_rate, 'tp':tp, 'fp':fp, 'fn':fn,  
            'precision':precision, 'recall':recall, 'f1':f1_score, 'iou':iou}

In [41]:
gt_file = open(root_dir+'0dtest.pkl', 'rb') #0dtest Nancho
gt_data = pickle.load(gt_file)

In [42]:
det_thres=  0.5 #0.35
data = []
pred_bbox =[]
target_bbox = []

try :
    del model
except NameError:
    pass

metric=MeanAveragePrecision(max_detection_thresholds=[1000])
model = init_detector(config_model, model_load)
for img_p in tqdm(gt_data):
    img = mmcv.imread(root_dir + 'test_images/'+img_p['filename'])
    pred = inference_detector(model, img)
    data.append(calc_conditions(img_p['filename'],img_p['ann']['bboxes'],pred[1], det_thres))
    #---get metrics
    pred_bbox.append(pred[1]) #[:,:-1]
    target_bbox.append(img_p['ann']['bboxes'])
    mpreds= [dict(
        boxes=torch.tensor(pred[1][:,:-1]),
        scores=torch.tensor(pred[1][:,-1]),
        labels=torch.tensor(np.ones(len(pred[1]),dtype=np.int32)),
        )]
    mtarget=[dict(
        boxes=torch.tensor(img_p['ann']['bboxes']),
        labels=torch.tensor(img_p['ann']['labels']),
        )]
    
    metric.update(mpreds, mtarget)
    # print(pred)
    # print("-----")
    # break
del model 
torch.cuda.empty_cache()
print('Donas!')

Fpc  ModuleList(
  (0): DeformConv2dPack(in_channels=256,
  out_channels=256,
  kernel_size=(3, 3),
  stride=(1, 1),
  padding=(1, 1),
  dilation=(1, 1),
  groups=1,
  deform_groups=1,
  bias=False)
  (1): DeformConv2dPack(in_channels=256,
  out_channels=256,
  kernel_size=(3, 3),
  stride=(1, 1),
  padding=(1, 1),
  dilation=(1, 1),
  groups=1,
  deform_groups=1,
  bias=False)
  (2): DeformConv2dPack(in_channels=256,
  out_channels=256,
  kernel_size=(3, 3),
  stride=(1, 1),
  padding=(1, 1),
  dilation=(1, 1),
  groups=1,
  deform_groups=1,
  bias=False)
  (3): DeformConv2dPack(in_channels=256,
  out_channels=256,
  kernel_size=(3, 3),
  stride=(1, 1),
  padding=(1, 1),
  dilation=(1, 1),
  groups=1,
  deform_groups=1,
  bias=False)
)
load checkpoint from local path: mmdetection/work_dirs/Nancho_dataset/defV2/thr_range[0_2]//epoch_20.pth


2023-03-19 11:28:25,745 - root - INFO - DeformConv2dPack neck.fpn_convs.0 is upgraded to version 2.
2023-03-19 11:28:25,748 - root - INFO - DeformConv2dPack neck.fpn_convs.1 is upgraded to version 2.
2023-03-19 11:28:25,751 - root - INFO - DeformConv2dPack neck.fpn_convs.2 is upgraded to version 2.
2023-03-19 11:28:25,753 - root - INFO - DeformConv2dPack neck.fpn_convs.3 is upgraded to version 2.
  0%|                                                                                            | 0/22 [00:00<?, ?it/s]/workspace/mmdetection_mau/code/mmdetection/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████| 22/22 [00:11<00:00,  1.93it/s]

Donas!


In [43]:
df = pd.DataFrame(data)
iou= np.average(df['iou'])
iou

0.5960101598719444

In [44]:
pprint(metric.compute())

{'map': tensor(-1.),
 'map_50': tensor(0.7069),
 'map_75': tensor(0.0556),
 'map_large': tensor(0.2148),
 'map_medium': tensor(0.2411),
 'map_per_class': tensor(-1.),
 'map_small': tensor(0.0274),
 'mar_1000': tensor(0.3189),
 'mar_1000_per_class': tensor(-1.),
 'mar_large': tensor(0.3000),
 'mar_medium': tensor(0.3418),
 'mar_small': tensor(0.0365)}


In [8]:
#kuzushiji 
# FPNimage.png
# {'map': tensor(-1.),
#  'map_50': tensor(0.8948),
#  'map_75': tensor(0.7596),
#  'map_large': tensor(0.7254),
#  'map_medium': tensor(0.5907),
#  'map_per_class': tensor(-1.),
#  'map_small': tensor(0.0079),
#  'mar_1000': tensor(0image.png.6974),
#  'mar_1000_per_class': tensor(-1.),
#  'mar_large': tensor(0.8266),
#  'mar_medium': tensor(0.6512),
#  'mar_small': tensor(0.0051)}
#def FPN
# {'map': tensor(-1.),
#  'map_50': tensor(0.9674),
#  'map_75': tensor(0.8703),
#  'map_large': tensor(0.7868),
#  'map_medium': tensor(0.7135),
#  'map_per_class': tensor(-1.),
#  'map_small': tensor(0.2985),
#  'mar_1000': tensor(0.7969),
#  'mar_1000_per_class': tensor(-1.),
#  'mar_large': tensor(0.8660),
#  'mar_medium': tensor(0.7728),
#  'mar_small': tensor(0.3729)}

In [7]:
#Nancho dataset
# def win_size=5, and scale=4
# {'map': tensor(-1.),
#  'map_50': tensor(0.9102),
#  'map_75': tensor(0.2147),
#  'map_large': tensor(0.4084),
#  'map_medium': tensor(0.3987),
#  'map_per_class': tensor(-1.),
#  'map_small': tensor(0.2300),
#  'mar_1000': tensor(0.4764),
#  'mar_1000_per_class': tensor(-1.),
#  'mar_large': tensor(0.4871),
#  'mar_medium': tensor(0.4915),
#  'mar_small': tensor(0.2863)}

In [8]:
df = pd.DataFrame(data)

tp=sum(df['tp'])
fp=sum(df['fp'])
fn=sum(df['fn'])
fppc=fp/(tp+fn)
dr=tp/(tp+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = (2 * precision * recall) / (precision + recall)
iou= np.average(df['iou'])
ap_101= average_precision(df['recall'],df['precision'],'101points')

new= {'image_id':'promedio', 'fppc':fppc, 'det rate':dr, 'tp':tp, 'fp':fp, 'fn':fn, 
        'precision':precision, 'recall':recall, 'f1':f1, 'iou':iou}  #, 'mAP101':ap_101
df= df.append(new, ignore_index = True)
df

class:  1


/tmp/ipykernel_2041977/2110644519.py:48: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  recalls = recalls[np.newaxis, :]
/tmp/ipykernel_2041977/2110644519.py:49: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  precisions = precisions[np.newaxis, :]


,image_id,fppc,det rate,tp,fp,fn,precision,recall,f1,iou
0,test_1.jpg,0.078120,0.932290,179,15,13,0.922680,0.932292,0.927461,0.864734
1,test_24.jpg,0.067620,0.907470,255,19,26,0.930657,0.907473,0.918919,0.850000
2,test_53.jpg,0.132650,0.892860,175,26,21,0.870647,0.892857,0.881612,0.788288
3,test_75.jpg,0.254550,0.642420,106,42,59,0.716216,0.642424,0.677316,0.512077
4,test_48.jpg,0.098360,0.939890,172,18,11,0.905263,0.939891,0.922252,0.855721
5,test_61.jpg,0.500000,0.916670,11,6,1,0.647059,0.916667,0.758621,0.611111
6,test_73.jpg,0.148150,0.944440,51,8,3,0.864407,0.944444,0.902655,0.822581
7,test_8.jpg,0.093530,0.949640,132,13,7,0.910345,0.949640,0.929577,0.868421
8,test_21.jpg,0.096770,0.749380,302,39,101,0.885630,0.749380,0.811828,0.683258
9,test_62.jpg,0.571430,0.857140,12,8,2,0.600000,0.857143,0.705882,0.545455


In [9]:
ap_area= average_precision(df['recall'][:-1],df['precision'][:-1],'area')
ap_11= average_precision(df['recall'][:-1],df['precision'][:-1],'11points')
ap_101= average_precision(df['recall'][:-1],df['precision'][:-1],'101points')
print('mAP_Area:', ap_area)
print('mAP11:', ap_11)
print('mAP101:', ap_101)

class:  1
class:  1
class:  1
mAP_Area: 0.92778826
mAP11: 0.8649603
mAP101: 0.08478324


/tmp/ipykernel_2041977/2110644519.py:48: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  recalls = recalls[np.newaxis, :]
/tmp/ipykernel_2041977/2110644519.py:49: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  precisions = precisions[np.newaxis, :]


In [11]:
pred_bbox[0][0][-1]

0.9987238

In [12]:
#df['precision'][-1:]
#def FPN
# mAP_Area: 0.92455816
# mAP11: 0.959152
# mAP101: 0.08586015
sizes_test =[[],[],[]]
sizes_score =[[],[],[]]

for ima in pred_bbox:
    for i in ima:
        letter_size=(i[2]-i[0])*(i[3]-i[1])
        if letter_size < 32**2:
            sizes_test[0].append(letter_size)
            sizes_score[0].append(i[-1])
        elif (letter_size>32**2) and (letter_size<96**2):
            sizes_test[1].append(letter_size)
            sizes_score[1].append(i[-1])
        elif letter_size > 96**2:
            sizes_test[2].append(letter_size)
            sizes_score[2].append(i[-1])


In [20]:
min(sizes_test[0])

122.51455

In [23]:
max(sizes_test[2])

70779.53

In [17]:
for i in sizes_test:
    print(len(i))
print('--Sizes--')
for i in sizes_score:
    print(np.average(i))

4064
89023
18696
--Sizes--
0.73534626
0.9604286
0.9733216


In [16]:
sizes_target =[[],[],[]]
for ima in target_bbox:
    for i in ima:
        letter_size=(i[2]-i[0])*(i[3]-i[1])
        if letter_size < 32**2:
            sizes_target[0].append(letter_size)
        elif (letter_size>32**2) and (letter_size<96**2):
            sizes_target[1].append(letter_size)
        elif letter_size > 96**2:
            sizes_target[2].append(letter_size)

for i in sizes_target:
    print(len(i))

4547
87840
18472


In [1]:
min(sizes_target[2])

NameError: name 'sizes_target' is not defined

In [ ]:
calc_conditions(img_p['filename'], img_p['ann']['bboxes'], pred[1], 0.5)

In [11]:
det_thres

0.5

### Save data??

In [12]:
df.to_csv(save_path+str(det_thres)+"_statistics.csv", index=False)

In [14]:
# --eval-options iou_thr=0.5
det_thres=  0.5 #0.35, 0.5
# data = []
try :
    del model
except NameError:
    pass

model = init_detector(config_model, model_load)
for img_p in tqdm(gt_data):
    img = mmcv.imread(root_dir + 'test_images/'+img_p['filename'])
    pred = inference_detector(model, img)
    labs = np.ones(len(pred[1]),dtype=str)
    # data.append(calc_conditions(img_p['filename'],img_p['ann']['bboxes'],pred[1], det_thres))
    image = mmcv.visualization.imshow_bboxes(img,img_p['ann']['bboxes'],thickness=2, show=False)
    mmcv.visualization.imshow_det_bboxes(image, pred[1], labels= labs, thickness=2, score_thr= det_thres,
                                        bbox_color='red',text_color='red',font_scale=0.02, 
                                        out_file=save_path+str(det_thres)+'_shows/'+img_p['filename'],show=False)
    # print("-----")
print("done!")
del model
torch.cuda.empty_cache()

Fpc  ModuleList(
  (0): DeformConv2dPack(in_channels=256,
  out_channels=256,
  kernel_size=(3, 3),
  stride=(1, 1),
  padding=(1, 1),
  dilation=(1, 1),
  groups=1,
  deform_groups=1,
  bias=False)
  (1): DeformConv2dPack(in_channels=256,
  out_channels=256,
  kernel_size=(3, 3),
  stride=(1, 1),
  padding=(1, 1),
  dilation=(1, 1),
  groups=1,
  deform_groups=1,
  bias=False)
  (2): DeformConv2dPack(in_channels=256,
  out_channels=256,
  kernel_size=(3, 3),
  stride=(1, 1),
  padding=(1, 1),
  dilation=(1, 1),
  groups=1,
  deform_groups=1,
  bias=False)
  (3): DeformConv2dPack(in_channels=256,
  out_channels=256,
  kernel_size=(3, 3),
  stride=(1, 1),
  padding=(1, 1),
  dilation=(1, 1),
  groups=1,
  deform_groups=1,
  bias=False)
)
load checkpoint from local path: mmdetection/work_dirs/HanDataset/top_best_mAP_epoch_5.pth


2023-02-14 19:06:07,359 - root - INFO - DeformConv2dPack neck.fpn_convs.0 is upgraded to version 2.
2023-02-14 19:06:07,360 - root - INFO - DeformConv2dPack neck.fpn_convs.1 is upgraded to version 2.
2023-02-14 19:06:07,360 - root - INFO - DeformConv2dPack neck.fpn_convs.2 is upgraded to version 2.
2023-02-14 19:06:07,361 - root - INFO - DeformConv2dPack neck.fpn_convs.3 is upgraded to version 2.
  0%|          | 0/350 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
100%|██████████| 350/350 [02:56<00:00,  1.99it/s]

done!


In [ ]:
from mmdet.core.evaluation.mean_ap import tpfp_default
tpp, fpp = tpfp_default(pred[1],img_p['ann']['bboxes'],img_p['ann']['bboxes_ignore'], iou_thr=0.35)
print(sum(sum(tpp)), sum(sum(fpp)))

In [ ]:
# save predictions
#pred
#mmcv.dump(pred, save_path+'predic.pkl')

Analize image cases

In [ ]:
import torch
import numpy as np
import pandas as pd
import PIL
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
import mmcv

import pickle
import albumentations as A
import cv2
import matplotlib.pyplot as plt
import cv2

# import required functions, classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.predict import get_prediction, get_sliced_prediction, predict
from IPython.display import Image
%matplotlib inline

dir = 'S05/swin_t'
model_name = 'top_cross_val_swin_nancho_defor.py'
model_load = 'mmdetection/work_dirs/'+dir+'/top_best_mAP_epoch_10.pth'
# model_load = 'mmdetection/work_dirs/'+'S03/swin_t'+'/top_best_mAP_epoch_12.pth' #usa S03 as model

root_dir = '/home/mauricio/Documents/Pytorch/mmdetection/mmdetection_mau/data/S05_Detection&Recognition/'  #Nancho_dataset
config_model = 'mmdetection/work_dirs/'+dir+'/'+model_name 
save_path = '/home/mauricio/Documents/Pytorch/mmdetection/mmdetection_mau/code/mmdetection/work_dirs/'+dir+'/'

gt_file = open(root_dir+'dtest.pkl', 'rb')
gt_data = pickle.load(gt_file)

In [ ]:
def crop2(img, w=5000, h=3000):
    center = img.shape
    x = center[1]/2 - w/2
    y = center[0]/2 - h/2
    # print(x,y)
    img_cropped = img[int(y):int(y+h), int(x):int(x+w)]
    return img_cropped

In [ ]:
transform = A.Compose(
    # [A.RandomResizedCrop(height=3300, width=3200, p=1)],
    [A.Crop(1000, 200,5000,3600, p=1)],
    bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.3, label_fields=['category_ids'])) #min_area=4500,

# transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids) # category_ids=category_ids
# visualize(
#     transformed['image'],
#     transformed['bboxes'],
#     transformed['category_ids'],
#     category_id_to_name,
# )

In [ ]:
pred[1]

In [ ]:
# --eval-options iou_thr=0.5
det_thres=  0.5 #0.35, 0.5
data = []
try :
    del model
except NameError:
    pass
   
model = init_detector(config_model, model_load)
for img_p in gt_data:
    if img_p['filename']== 'S05_027.jpg':
        img = mmcv.imread(root_dir + 'test_images/'+img_p['filename']) #
        bboxes = img_p['ann']['bboxes']  #albu
        category_ids= img_p['ann']['labels'] #albu
        # img = transform(image=img, bboxes=bboxes, category_ids=category_ids) #albu
        # img= crop2(img, 2800, 3200)
        pred = inference_detector(model, img) #img
        # pred = inference_detector(model, img['image']) #albu
        labs = np.ones(len(pred[1]),dtype=str)
        # ###data.append(calc_conditions(img_p['filename'],img_p['ann']['bboxes'],pred[1], det_thres))
        result= calc_conditions(img_p['filename'], img_p['ann']['bboxes'], pred[1], det_thres)  
        image = mmcv.visualization.imshow_bboxes(img,img_p['ann']['bboxes'],thickness=2, show=False)
        # result= calc_conditions(img_p['filename'], img['bboxes'], pred[1], det_thres)  #albu 
        # image = mmcv.visualization.imshow_bboxes(img['image'],np.array(img['bboxes']), thickness=2, show=False) #albu
        mmcv.visualization.imshow_det_bboxes(image, pred[1], labels= labs, thickness=2, score_thr= det_thres,
                                            bbox_color='red',text_color='red',font_scale=0.02, 
                                            out_file=save_path+'demos/'+str(det_thres)+'_'+img_p['filename'], show=False) #
        # print("-----")
del model
torch.cuda.empty_cache()
print(result)
print("Done!!")

In [ ]:
##Sliced inference using sahi 
spec_img= 'S05_027'
detection_model = AutoDetectionModel.from_pretrained(
    model_type='mmdet',
    model_path=model_load,
    config_path=config_model,
    confidence_threshold=det_thres,
    image_size=1200,
    device="cuda:0", # 'cpu' or 'cuda:0'
)
# result = get_prediction(root_dir + 'test_images/'+spec_img+'.jpg', detection_model)
# result.export_visuals(export_dir=save_path+'demos/sahi_'+str(det_thres)+'_'+spec_img+'.jpg')

result = get_sliced_prediction(
    root_dir + 'test_images/'+spec_img+'.jpg',
    detection_model,
    slice_height = 1000,
    slice_width = 1000,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2
)
# result.export_visuals(export_dir=save_path+'demos/sahi_slided_'+str(det_thres)+'_'+spec_img+'.jpg')
print('donas')

In [ ]:
slided_pred= []
# for i in result1.to_coco_annotations():
#     slided_pred.append(np.array((*i['bbox'],i['score'])))
# slided_pred=np.array((slided_pred))
for i in result.object_prediction_list:
    slided_pred.append(np.array((i.bbox.minx,i.bbox.miny,i.bbox.maxx, i.bbox.maxy, i.score.value)))
slided_pred=np.array((slided_pred))

In [ ]:
for img_p in gt_data:
    if img_p['filename']== spec_img+'.jpg':
        img = mmcv.imread(root_dir + 'test_images/'+img_p['filename']) #
        labs = np.ones(len(slided_pred),dtype=str)
        image = mmcv.visualization.imshow_bboxes(img,img_p['ann']['bboxes'],thickness=2, show=False)
        mmcv.visualization.imshow_det_bboxes(image, slided_pred, labels= labs, thickness=2, score_thr= det_thres,
                                            bbox_color='red',text_color='red',font_scale=0.02, 
                                            out_file=save_path+'demos/sahi_'+str(det_thres)+'_'+img_p['filename'], show=False) #

In [ ]:
# aa= result1.object_prediction_list[0]
# print(list((aa.bbox.minx,aa.bbox.miny,aa.bbox.maxx, aa.bbox.maxy, aa.score.value)))

In [ ]:
#using S05 model:
1280, 0.5 (00_)
{'image_id': 'S05_027.jpg', 'fppc': 0.23767, 'det rate': 0.10762, 'tp': 24, 'fp': 53, 'fn': 199, 'precision': 0.3116883116883117, 'recall': 0.10762331838565023, 'f1': 0.16}
1500, 0.5 (002_)
{'image_id': 'S05_027.jpg', 'fppc': 0.20179, 'det rate': 0.4574, 'tp': 102, 'fp': 45, 'fn': 121, 'precision': 0.6938775510204082, 'recall': 0.45739910313901344, 'f1': 0.5513513513513514}


#using S03 model:
0.1{'image_id': 'S05_027.jpg', 'fppc': 0.05381, 'det rate': 0.83857, 'tp': 187, 'fp': 12, 'fn': 36, 'precision': 0.9396984924623115, 'recall': 0.8385650224215246, 'f1': 0.8862559241706162}
0.5{'image_id': 'S05_027.jpg', 'fppc': 0.1704, 'det rate': 0.55605, 'tp': 124, 'fp': 38, 'fn': 99, 'precision': 0.7654320987654321, 'recall': 0.5560538116591929, 'f1': 0.6441558441558441}

In [ ]:
calc_conditions(img_p['filename'], img_p['ann']['bboxes'], pred[1], 0.5)

In [ ]:
image.shape

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
def crop2(img, w=5000, h=3000):
    center = img.shape
    x = center[1]/2 - w/2
    y = center[0]/2 - h/2
    # print(x,y)
    img_cropped = img[int(y):int(y+h), int(x):int(x+w)]
    return img_cropped

In [ ]:
cropped= crop2(image, 2800, 3200)
plt.figure(figsize=(15, 15))
plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
plt.figure(figsize=(15, 15))
plt.hist(image.ravel(),256,[0,256]); plt.show()

In [ ]:
unique, counts = np.unique(image, return_counts=True)

dict(zip(unique, counts))

In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White

def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(w), int(y_min), int(h)
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img

def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(img)
    

In [ ]:
category_id_to_name = {0: 'background', 1: 'cc'}
for img_p in gt_data:
    if img_p['filename']== 'S05_027.jpg':
        # print(img_p)
        image = cv2.imread(root_dir + 'test_images/'+img_p['filename'])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        bboxes = img_p['ann']['bboxes']
        category_ids= img_p['ann']['labels']
        # img = mmcv.imread(root_dir + 'test_images/'+img_p['filename']) #

In [ ]:
len(category_ids)
# len(bboxes)

In [ ]:
# transform = A.Compose(
#     [A.ShiftScaleRotate(p=0.5)],
#     bbox_params=A.BboxParams(format='pascal_voc',label_fields=['category_ids']),
# )
transform = A.Compose(
    # [A.RandomResizedCrop(height=3300, width=3200, p=1)],
    [A.Crop(1000, 200,5000,3600, p=1)],
    bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.3, label_fields=['category_ids'])) #min_area=4500,

transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids) # category_ids=category_ids
visualize(
    transformed['image'],
    transformed['bboxes'],
    transformed['category_ids'],
    category_id_to_name,
)
# visualize(
#     image,
#     bboxes,
#     category_ids,
#     category_id_to_name
# )

In [ ]:
pred[1][:,:-1] #get bboxes
(pred[1][:,-1]) #get scores
#len(img['bboxes']) #get gt bboxes
len(img['category_ids']) #get labels

In [ ]:
# labels_pred = [
#     np.full(bbbox.shape[0], i, dtype=np.int32)\
#     for i, bbbox in enumerate(pred)
# ]
#pred
tboxes= torch.tensor(pred[1][:,:-1])
tscores= torch.tensor(pred[1][:,-1])
tlabels= torch.tensor(np.ones(len(pred[1]),dtype=np.int32))
#ground True
tgt_boxes= torch.tensor(img_p['ann']['bboxes']) #img['bboxes']
tgt_labels= torch.tensor(img_p['ann']['labels']) #img['category_ids']

In [ ]:
import torch
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from pprint import pprint

mpreds= [dict(
    boxes=torch.tensor(pred[1][:,:-1]),
    scores=torch.tensor(pred[1][:,-1]),
    labels=torch.tensor(np.ones(len(pred[1]),dtype=np.int32)),
)]
mtarget=[dict(
    boxes=torch.tensor(img_p['ann']['bboxes']),
    labels=torch.tensor(img_p['ann']['labels']),
)]
metric=MeanAveragePrecision()
metric.update(mpreds, mtarget)
metric.compute()

In [ ]:
len(pred[1])

In [ ]:
metric = MeanAveragePrecision()

In [ ]:
#demo 2
preds = [
  dict(
    boxes=torch.tensor([[158.0, 21.0, 606.0, 285.0]]),
    scores=torch.tensor([0.546]),
    labels=torch.tensor([0]),
  )
]
target = [
  dict(
    boxes=torch.tensor([[22.0, 41.0, 662.0, 345.0]]),
    labels=torch.tensor([0]),
  )
]

metric.update(preds, target)




In [ ]:
metric.detection_scores

In [ ]:
pprint(metric.compute())

In [ ]:
{'map': tensor(0.6000),
 'map_50': tensor(1.),
 'map_75': tensor(1.),
 'map_large': tensor(0.6000),
 'map_medium': tensor(-1.),
 'map_per_class': tensor(-1.),
 'map_small': tensor(-1.),
 'mar_1': tensor(0.6000),
 'mar_10': tensor(0.6000),
 'mar_100': tensor(0.6000),
 'mar_100_per_class': tensor(-1.),
 'mar_large': tensor(0.6000),
 'mar_medium': tensor(-1.),
 'mar_small': tensor(-1.)}